In [ ]:
# 실행하고 링크 누르기
# 런타임 유형 v2-8 TPU

In [ ]:
print("Setting up GCS access...")
import os
os.environ['USE_AUTH_EPHEM'] = '0'
from google.colab import auth
auth.authenticate_user()

Setting up GCS access...


In [ ]:
##버전은 언제든지 변경이 될 수 있음. 라이브러리 문제. 아래 cell에서 import 되지 않으면 맞는 버전 찾기
from IPython.display import clear_output
!pip install gcsfs
!pip install https://storage.googleapis.com/cloud-tpu-tpuvm-artifacts/tensorflow/tf-2.15.0/tensorflow-2.15.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip install tensorflow
!pip install t5
#!pip install 'jax[tpu]==0.4.26' -f https://storage.googleapis.com/jax-releases/libtpu_releases.html -f https://storage.googleapis.com/jax-releases/jax_releases.html
#!pip install jaxlib==0.4.26
!pip install tensorflow-gcs-config==2.15.0
!pip install -q tensorflow-text==2.15.0
clear_output()
print("Installing dependencies...")

Installing dependencies...


In [ ]:
import functools
import os
import gin
from contextlib import contextmanager
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import tensorflow.compat.v1 as tf
import tensorflow_datasets as tfds
import logging as py_logging
import t5

In [ ]:
tf.app.flags.DEFINE_string ('f', '', '')

In [ ]:
import tensorflow_gcs_config
import tensorflow.compat.v1 as tf
TPU_TOPOLOGY = "2x2"

try:
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='local')  # TPU detection
    tf.config.experimental_connect_to_cluster(resolver)
    tf.config.experimental.list_logical_devices('TPU')
    tf.config.experimental.list_physical_devices('TPU')
    print("All devices: ", tf.config.list_logical_devices('TPU'))
    s = tf.Session()
    print(s.list_devices())
    tensorflow_gcs_config.configure_gcs_from_colab_auth(tf.config.list_logical_devices('TPU')[1])
except ValueError:
    raise BaseException(
        'ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!'
    )


tf.disable_v2_behavior()

#LOGGING
tf.get_logger().propagate = False
py_logging.root.setLevel('INFO')

@contextmanager
def tf_verbosity_level(level):
  og_level = tf.logging.get_verbosity()
  tf.logging.set_verbosity(level)
  yield
  tf.logging.set_verbosity(og_level)

Instructions for updating:
non-resource variables are not supported in the long term


All devices:  [LogicalDevice(name='/device:TPU:0', device_type='TPU'), LogicalDevice(name='/device:TPU:1', device_type='TPU'), LogicalDevice(name='/device:TPU:2', device_type='TPU'), LogicalDevice(name='/device:TPU:3', device_type='TPU'), LogicalDevice(name='/device:TPU:4', device_type='TPU'), LogicalDevice(name='/device:TPU:5', device_type='TPU'), LogicalDevice(name='/device:TPU:6', device_type='TPU'), LogicalDevice(name='/device:TPU:7', device_type='TPU')]
[_DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 268435456, -5315752443649472665), _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:0, TPU, 17179869184, 27174686991857553), _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:1, TPU, 17179869184, -2330879005499457346), _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:2, TPU, 17179869184, -8475818048530131393), _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:3, TPU, 17179869184, -7821953537311121992), _DeviceAttribute

In [ ]:
representation = "custom" #@param ['tokens','custom'] #tokens은 leonid, custom은 우리가 만든 데이터셋
pretraining_task = "word_real" #@param ['token_real','word_real'] #word_real은 word 기준인 pretrained model 경로, token_real은 token 기준인 pretrined model 경로
scheduler = "isr" #@param ["polynomial", "constant", "isr", "slanted"]
my_finetuning_task = "multi-log-injection" #@param ['multi-log-injection', 'single-log-injection', 'classifier'] #multi-log injection은 다중 로그 삽입, classifier은 로그 삽입 여부
multi_log_injection_type = "one-to-n" #@param [ "one-to-n"]
s_TOP = 'TOP-5' #@param['TOP-1','TOP-3','TOP-5']
m_TOP = 'TOP5' #@param['TOP1','TOP3','TOP5']
prefix = 'MULTI_LOG_INJECTION: ' #@param ['SINGLE_LOG_INJECTION: ', 'CLASSIFICATION: ', 'MULTI_LOG_INJECTION: ']
TASK_NAME = "log_injection" #@param ['classification', 'log_injection'] #log_injection은 로그 삽입 메세지 생성, classification은 로그 삽입 여부 결정

In [ ]:
if my_finetuning_task == "multi-log-injection":
  train_path = f'gs://log_gen/finetune/{my_finetuning_task}/{multi_log_injection_type}/{representation}/{m_TOP}/train_add_token.tsv'
  eval_path = f'gs://log_gen/finetune/{my_finetuning_task}/{multi_log_injection_type}/{representation}/{m_TOP}/eval_add_token.tsv'
  test_path = f'gs://log_gen/finetune/{my_finetuning_task}/{multi_log_injection_type}/{representation}/{m_TOP}/test_add_token.tsv'
elif my_finetuning_task =="single-log-injection":
  train_path = f'gs://log_gen/finetune/{my_finetuning_task}/{representation}/{s_TOP}/train.tsv'
  eval_path = f'gs://log_gen/finetune/{my_finetuning_task}/{representation}/{s_TOP}/eval.tsv'
  test_path = f'gs://log_gen/finetune/{my_finetuning_task}/{representation}/{s_TOP}/test.tsv'
else:
  train_path = f'gs://log_gen/finetune/{my_finetuning_task}/train.tsv'
  eval_path = f'gs://log_gen/finetune/{my_finetuning_task}/eval.tsv'
  test_path = f'gs://log_gen/finetune/{my_finetuning_task}/test-75-25.tsv'

finetune_datasets_paths = {
    "train":      train_path,
    "validation": eval_path,
    "test": test_path
}
train_path
eval_path
test_path

'gs://log_gen/finetune/multi-log-injection/one-to-n/custom/TOP5/test_add_token.tsv'

In [ ]:
# Storage paths
if my_finetuning_task == "multi-log-injection":
  FINETUNE_MODEL_DIR = f"gs://log_gen/finetuned-model/{my_finetuning_task}/{multi_log_injection_type}/{representation}/word_fixed" #WORD PRETRAIN 모델을 사용하면 word_fixed, TOKEN PRETRAIN 모델을 사용하면 token_fixed
elif my_finetuning_task == "single-log-injection":
  FINETUNE_MODEL_DIR = f"gs://log_gen/finetuned-model/{my_finetuning_task}/{representation}/token"
else:
  FINETUNE_MODEL_DIR = f"gs://log_gen/finetuned-model/{my_finetuning_task}/token/"

PRETRAIN_MODEL_DIR= f'gs://log_gen/pretrained-model/custom/{pretraining_task}'
FINETUNE_MODEL_DIR
#PRETRAIN_MODEL_DIR

'gs://log_gen/finetuned-model/multi-log-injection/one-to-n/custom/word_fixed'

In [ ]:
vocab_model_path = f"gs://log_gen/tokenizer/tokens/tokenizer.model"
vocab_path = f"gs://log_gen/tokenizer/tokens/tokenizer.vocab"

In [ ]:
from t5.data import postprocessors as t5_postprocessors
from t5.seqio import Feature,SentencePieceVocabulary

num_special_mask_tokens = 100 #@param {type: "integer"}

def load_vocabulary():
  return SentencePieceVocabulary(vocab_model_path, num_special_mask_tokens)

In [ ]:
BASE_DIR = f"gs://log_gen/learning_rate_scheduler/{scheduler}"
print(BASE_DIR)
GIN_FILENAME = "isr.gin" #@param {type : "polynomial","isr"}#
# 'operative_config.gin'
# 'no_pretraining_operative_config.gin'
remote_gin_path = os.path.join(BASE_DIR, GIN_FILENAME)
LOCAL_GIN_PATH = f"/content/{GIN_FILENAME}"
!gsutil -m cp $remote_gin_path $LOCAL_GIN_PATH

gs://log_gen/learning_rate_scheduler/isr
Copying gs://log_gen/learning_rate_scheduler/isr/isr.gin...
/ [1/1 files][ 11.5 KiB/ 11.5 KiB] 100% Done                                    
Operation completed over 1 objects/11.5 KiB.                                     


In [ ]:
"""AttributeError                            Traceback (most recent call last)
<ipython-input-13-efe9cf962c6f> in <cell line: 1>()
      1 with gin.unlock_config():
----> 2     gin.parse_config_file(LOCAL_GIN_PATH)

10 frames
/usr/local/lib/python3.10/dist-packages/mesh_tensorflow/tpu_variables.py in <module>
    224
    225
--> 226 ops.register_tensor_conversion_function(ReplicatedVariable, _tensor_conversion)
    227
    228 if not TF_23:

AttributeError: module 'tensorflow.python.framework.ops' has no attribute 'register_tensor_conversion_function'
  In file "/content/isr.gin", line 1
    import mesh_tensorflow.optimize"""
#위와 같은 에러가 아래 cell에서 발생 가능

# 에러에서 "/usr/local/lib/python3.10/dist-packages/mesh_tensorflow/tpu_variables.py/usr/local/lib/python3.10/dist-packages/mesh_tensorflow/tpu_variables.py"를 누르면 tpu_variable.py가 옆에 나옴
# tpu_variable.py 226줄에서 "ops.register_tensor_conversion_function(ReplicatedVariable, _tensor_conversion)" 를
# ops.tensor_conversion_registry.register_tensor_conversion_function(ReplicatedVariable, _tensor_conversion) 로 변경

In [ ]:
with gin.unlock_config():
    gin.parse_config_file(LOCAL_GIN_PATH)

In [ ]:
def load_dataset(split, shuffle_files=True):
  del shuffle_files

  ds = tf.data.TextLineDataset(finetune_datasets_paths[split])
  #ds = ds.skip(42)
  def has_two_fields(line):
        return tf.equal(tf.size(tf.strings.split([line], sep="\t")), 2)

  ds = ds.filter(has_two_fields)
  ds = ds.map(functools.partial(tf.io.decode_csv, record_defaults=["string","string"],
                          field_delim="\t", use_quote_delim=False),
        num_parallel_calls=tf.data.experimental.AUTOTUNE)

  ds = ds.map(lambda *ex: dict(zip(["input", "output"], ex)))
  def truncate_examples(ex):
        ex['input'] = tf.strings.substr(ex['input'], 0, 2048)
        ex['output'] = tf.strings.substr(ex['output'], 0, 2048)
        return ex

  ds = ds.map(truncate_examples)

  return ds

In [ ]:
print("A few raw validation examples...")
for ex in tfds.as_numpy(load_dataset("train").take(5)):
  print(ex)

A few raw validation examples...
{'input': b'augmented_input', 'output': b'original_method'}
{'input': b'"protected String[] listSupportedSASLMechanisms(AtomicBoolean done) { final CountDownLatch listMechsLatch = new CountDownLatch(1); final AtomicReference<String> supportedMechs = new AtomicReference<String>(); Operation listMechsOp = opFact.saslMechs(new OperationCallback() { @Override public void receivedStatus(OperationStatus status) { if(status.isSuccess()) { supportedMechs.set(status.getMessage()); getLogger().debug(""Received SASL supported mechs: "" + status.getMessage()); } else { getLogger().warn(""Received non-success response for SASL mechs: "" + status); } } @Override public void complete() { listMechsLatch.countDown(); } }); conn.insertOperation(node, listMechsOp); try { if (!conn.isShutDown()) { listMechsLatch.await(); } else { done.set(true); // Connection is shutting down, tear.down. } } catch(InterruptedException ex) { getLogger().warn(""Interrupted in Auth while wait

In [ ]:
from tensorflow_datasets.core.utils.type_utils import Shape
def preprocessing(ds):
  def to_inputs_and_targets(ex):
    x_input = tf.strings.strip(prefix + ex['input'])
    y_label = tf.strings.strip(ex['output'])
    inputs = tf.strings.join([x_input], separator=' ')
    class_label = tf.strings.join([y_label], separator=' ')
    return {'inputs': inputs, 'targets': class_label}
  return ds.map(to_inputs_and_targets,
                num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [ ]:
print("A few preprocessed train examples...")
sample = tfds.as_numpy(preprocessing(load_dataset("train").take(5)))
for ex in sample:
  print(ex)

A few preprocessed train examples...
{'inputs': b'MULTI_LOG_INJECTION: augmented_input', 'targets': b'original_method'}
{'inputs': b'MULTI_LOG_INJECTION: "protected String[] listSupportedSASLMechanisms(AtomicBoolean done) { final CountDownLatch listMechsLatch = new CountDownLatch(1); final AtomicReference<String> supportedMechs = new AtomicReference<String>(); Operation listMechsOp = opFact.saslMechs(new OperationCallback() { @Override public void receivedStatus(OperationStatus status) { if(status.isSuccess()) { supportedMechs.set(status.getMessage()); getLogger().debug(""Received SASL supported mechs: "" + status.getMessage()); } else { getLogger().warn(""Received non-success response for SASL mechs: "" + status); } } @Override public void complete() { listMechsLatch.countDown(); } }); conn.insertOperation(node, listMechsOp); try { if (!conn.isShutDown()) { listMechsLatch.await(); } else { done.set(true); // Connection is shutting down, tear.down. } } catch(InterruptedException ex) { 

In [ ]:
DEFAULT_OUTPUT_FEATURES = {
    "inputs": Feature(
        vocabulary=load_vocabulary(), add_eos=True, required=False),
    "targets": Feature(
        vocabulary=load_vocabulary(), add_eos=True)
    }

# TASK
t5.data.TaskRegistry.remove(TASK_NAME)
t5.data.TaskRegistry.add(
    TASK_NAME,
    dataset_fn=load_dataset,
    splits=["train",
            "validation",
            "test"
            ],
    text_preprocessor=[preprocessing],
    metric_fns=[t5.evaluation.metrics.accuracy],
    output_features = DEFAULT_OUTPUT_FEATURES
)

MIXTURE_NAME = "finetuning" #@param{ type : "string"}

# MIXTURE
t5.data.MixtureRegistry.remove(MIXTURE_NAME)
t5.data.MixtureRegistry.add(
    MIXTURE_NAME,
    [TASK_NAME],
    default_rate=1.0
)

In [ ]:
finetuning_task = t5.data.TaskRegistry.get(TASK_NAME)
ds = finetuning_task.get_dataset(split="train", sequence_length={"inputs": 512, "targets": 512})
print("A few preprocessed training examples...")
for ex in tfds.as_numpy(ds.take(5)):
  print(ex)

A few preprocessed training examples...
{'inputs_pretokenized': b'MULTI_LOG_INJECTION: "@Override public boolean schemaExists(ConnectorSession session, String schemaName) { BigQueryClient client = bigQueryClientFactory.create(session); DatasetId localDatasetId = client.toDatasetId(schemaName); // Overridden to make sure an error message is returned in case of an ambiguous schema name <LOG> return client.toRemoteDataset(localDatasetId) .map(RemoteDatabaseObject::getOnlyRemoteName) .filter(remoteSchema -> client.getDataset(DatasetId.of(localDatasetId.getProject(), remoteSchema)) != null) .isPresent(); }<log_message> ""listTables(session=%s, schemaName=%s)"" </log_message>  <similarity> 0.35 </similarity><log_message> ""listTables(session=%s, schemaName=%s)"" </log_message>  <similarity> 0.35 </similarity><log_message> ""Filtered out [%s] from list of tables due to ambiguous name"" </log_message>  <similarity> 0.28 </similarity><log_message> ""Filtered out [%s] from list of tables due to 

In [ ]:
from tensorflow.keras.optimizers.schedules import PolynomialDecay

# Learning rate properties
starter_learning_rate = 0.01 #@param {type : "number"}
end_learning_rate = 0.001 #@param {type : "number"}
decay_steps = 10000 #@param {type : "integer"}

learning_rate_fn = PolynomialDecay(
     starter_learning_rate,
     decay_steps,
     end_learning_rate,
     power=0.5)

In [ ]:
from mesh_tensorflow.transformer.learning_rate_schedules import slanted_triangular, truncated_rsqrt
from t5 import models

if scheduler == 'polynomial':
  learning_rate_scheduler = learning_rate_fn
elif scheduler == 'isr':
  learning_rate_scheduler = truncated_rsqrt
elif scheduler == 'slanted':
  learning_rate_scheduler = slanted_triangular
else:
  learning_rate_scheduler = 0.001

print(learning_rate_scheduler)

MODEL_SIZE = "small"
model_parallelism, train_batch_size, keep_checkpoint_max = {
    "small": (1, 128, 100),
    "base": (2, 128, 8),
    "large": (8, 64, 4),
    "3B": (8, 16, 1),
    "11B": (8, 16, 1)}[MODEL_SIZE]


model = t5.models.MtfModel(
    model_dir=FINETUNE_MODEL_DIR,
    tpu='local',
    model_parallelism=model_parallelism,
    batch_size=train_batch_size,
    sequence_length={"inputs": 1024, "targets": 1024},
    learning_rate_schedule = learning_rate_scheduler,
    save_checkpoints_steps=10000,
    keep_checkpoint_max=keep_checkpoint_max
)

<function truncated_rsqrt at 0x7cf5247d6680>


In [ ]:
#체크 포인트가 있으면 로드
checkpoint_dir = FINETUNE_MODEL_DIR
latest_checkpoint = tf.train.latest_checkpoint(checkpoint_dir)
if latest_checkpoint:
    print(f"Resuming from checkpoint: {latest_checkpoint}")
else:
    print("No checkpoint found. Starting training from scratch.")

Resuming from checkpoint: gs://log_gen/finetuned-model/multi-log-injection/one-to-n/custom/word_fixed/model.ckpt-600000


In [ ]:
TRAIN_STEPS =  100000 #@param {type: "integer"}

# Stat finetuning
model.finetune(mixture_or_task_name=MIXTURE_NAME,
               finetune_steps=TRAIN_STEPS,
               pretrained_model_dir=FINETUNE_MODEL_DIR)#PRETRAIN_MODEL_DIR) #FINETUNE_MODEL_DIR
               #PRETRAIN_MODEL_DIR: 체크포인트 없을 때
               #FINETUNE_MODEL_DIR: 체크포인트 있을 때

INFO:root:system_path_file_exists:gs://log_gen/finetuned-model/multi-log-injection/one-to-n/custom/word_fixed/operative_config.gin
ERROR:root:Path not found: gs://log_gen/finetuned-model/multi-log-injection/one-to-n/custom/word_fixed/operative_config.gin
From /usr/local/lib/python3.10/dist-packages/mesh_tensorflow/transformer/utils.py:2043: TPUConfig.__new__ (from tensorflow_estimator.python.estimator.tpu.tpu_config) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.keras instead.
From /usr/local/lib/python3.10/dist-packages/mesh_tensorflow/transformer/utils.py:2059: RunConfig.__init__ (from tensorflow_estimator.python.estimator.tpu.tpu_config) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.keras instead.
From /usr/local/lib/python3.10/dist-packages/tensorflow_estimator/python/estimator/tpu/tpu_config.py:268: RunConfig.__init__ (from tensorflow_estimator.python.estimator.run_config) is deprecated and will 

In [ ]:
checkpoints = []
for check in range(500000, 600000, 10000):
  checkpoints.append(check)
checkpoints

[500000,
 510000,
 520000,
 530000,
 540000,
 550000,
 560000,
 570000,
 580000,
 590000]

In [ ]:
from t5 import models

model = t5.models.MtfModel(FINETUNE_MODEL_DIR, tpu='local',sequence_length={"inputs": 1024, "targets": 1024},
    learning_rate_schedule = learning_rate_scheduler,)
model.batch_size=16
model.eval(
    mixture_or_task_name=MIXTURE_NAME,
    checkpoint_steps=checkpoints
    split='test'
)

INFO:root:system_path_file_exists:gs://log_gen/finetuned-model/multi-log-injection/one-to-n/custom/word_fixed/operative_config.gin
ERROR:root:Path not found: gs://log_gen/finetuned-model/multi-log-injection/one-to-n/custom/word_fixed/operative_config.gin
INFO:absl:Adding task 'log_injection' with predict metric_fn(s).
INFO:absl:Skipping packing/padding for 'log_injection' since sequence length is None.
INFO:absl:Setting sequence lengths to {'inputs': 1377, 'targets': 1298}
INFO:absl:Evaluating checkpoint step: 600000
From /usr/local/lib/python3.10/dist-packages/mesh_tensorflow/transformer/utils.py:2043: TPUConfig.__new__ (from tensorflow_estimator.python.estimator.tpu.tpu_config) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.keras instead.
From /usr/local/lib/python3.10/dist-packages/mesh_tensorflow/transformer/utils.py:2059: RunConfig.__init__ (from tensorflow_estimator.python.estimator.tpu.tpu_config) is deprecated and will be removed in a f

In [ ]:
#MULTI-LOG INJECITON 예시출력용
import random

def print_random_predictions(task_name, TASK_NAME, my_finetuning_task, n=10):
  """Print n predictions from the validation split of a task."""
  # Grab the dataset for this task.
  finetuning_task = t5.data.TaskRegistry.get(TASK_NAME)
  ds = finetuning_task.get_dataset(split="test", sequence_length={"inputs": 1024, "targets": 1024},shuffle=False)

  def _prediction_file_to_ckpt(path):
    """Extract the global step from a prediction filename."""
    return int(path.split("_")[-2])

  # Grab the paths of all logged predictions.
  prediction_files = tf.io.gfile.glob(
      os.path.join(
          FINETUNE_MODEL_DIR,
          "test_eval/%s_*_predictions" % TASK_NAME ))
  print(os.path.join(
          FINETUNE_MODEL_DIR,
          "test_eval/%s_*_predictions" % TASK_NAME))
  print(prediction_files)
 # Get most recent prediction file by sorting by their step.
  latest_prediction_file = sorted(
      prediction_files, key=_prediction_file_to_ckpt)[-1]

  # Collect (inputs, targets, prediction) from the dataset and predictions file
  results = []
  with tf.io.gfile.GFile(latest_prediction_file) as preds:
    for ex, pred in zip(tfds.as_numpy(ds), preds):
      results.append((tf.compat.as_text(ex["inputs_pretokenized"]),
                      tf.compat.as_text(ex["targets_pretokenized"]),
                      pred.strip()))

  print("<== Random predictions for %s using checkpoint %s ==>\n" %
        (task_name,
         _prediction_file_to_ckpt(latest_prediction_file)))

  for inp, tgt, pred in random.choices(results, k=10):
    print("Input:", inp)
    print("Target:", tgt)
    print("Prediction:", pred)
    print("Counted as Correct?", tgt == pred)
    print()

print_random_predictions(MIXTURE_NAME,TASK_NAME,prefix)
#print_random_predictions("nq_context_free")


gs://log_gen/finetuned-model/multi-log-injection/one-to-n/custom/word_fixed/test_eval/log_injection_*_predictions
['gs://log_gen/finetuned-model/multi-log-injection/one-to-n/custom/word_fixed/test_eval/log_injection_600000_predictions']
<== Random predictions for finetuning using checkpoint 600000 ==>

Input: MULTI_LOG_INJECTION: "@Override public void didFinishLaunching() { if (ERXProperties.booleanForKey(""er.javamail.mailer.ERTestSendingMail"")); testSendingMail(); int frequency = ERXProperties.intForKey(""er.javamail.mailer.ERBatchMailerDaemonFrequency""); if (frequency > 0) { <LOG> mailTimer = new Timer(true); mailTimer.schedule(new ERMailerTimerTask(), frequency*1000l, frequency*1000l); } else { ERMailer.instance().processOutgoingMail(); <LOG> System.exit(0); } }<log_message> ""Creating poll-guard timer with interval ["" ""ms] while starting SpringJmsConnector"" </log_message>  <similarity> 0.24 </similarity><log_message> ""Preparing to shutdown simulator..."" </log_message>  <si

In [ ]:
!pip install nltk
!pip install rouge-score
!pip install tree_sitter
!pip install requests
!pip install tensorflow-text
!pip install seqeval
!pip install pygments
clear_output()
print("Installing dependencies...")

Installing dependencies...


In [ ]:
#!git clone https://github.com/tree-sitter/tree-sitter-java.git

!pip install GitPython

from git import Repo

repo_url = 'https://github.com/tree-sitter/tree-sitter-java.git'
clone_directory = '/content/CodeBLEU_2/parsers/tree-sitter-java'

if not os.path.exists(clone_directory):
    Repo.clone_from(repo_url, clone_directory)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.1 MB/s eta 0:00:00


In [ ]:
import sys
import os
import random
import nltk
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_text
import t5
import seqeval

sys.path.append('/content/CodeBLEU_2')

nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score
from rouge_score import rouge_scorer
#from CodeBLEU_2 import calc_code_bleu
from pygments.lexers import get_lexer_by_name
from pygments.token import Token
from pygments import lex

In [ ]:
import glob

def _prediction_file_to_ckpt(path):
    """Extract the global step from a prediction filename."""
    return int(path.split("_")[-2])

# 예측 결과 파일 경로 패턴
prediction_files_pattern = os.path.join(
    FINETUNE_MODEL_DIR,
    "test_eval",
    "%s_*_predictions" % TASK_NAME
)

In [ ]:
#input, reference, prediciton 예시 출력
prediction_files = tf.io.gfile.glob(prediction_files_pattern)

latest_prediction_file = sorted(
    prediction_files, key=_prediction_file_to_ckpt)[-1]

print("사용할 예측 결과 파일:", latest_prediction_file)

finetuning_task = t5.data.TaskRegistry.get(TASK_NAME)
ds = finetuning_task.get_dataset(
    split="test",
    sequence_length={"inputs": 1024, "targets": 1024},
    shuffle=False
)

results = []
with tf.io.gfile.GFile(latest_prediction_file) as preds:
    for ex, pred in zip(tfds.as_numpy(ds), preds):
        input_text = tf.compat.as_text(ex["inputs_pretokenized"])
        target_text = tf.compat.as_text(ex["targets_pretokenized"])
        prediction_text = pred.strip()
        results.append((input_text, target_text, prediction_text))

inputs, references, predictions = zip(*results)

print("Input:", inputs[2])
print("Reference:", references[2])
print("Prediction:", predictions[2])

사용할 예측 결과 파일: gs://log_gen/finetuned-model/multi-log-injection/one-to-n/custom/word_fixed/test_eval/log_injection_600000_predictions
Input: MULTI_LOG_INJECTION: "@Override public void doInTransactionWithoutResult(final TransactionStatus status) { if (addVnetsFinal != null) { <LOG> //add vnet takes a list of strings to be added. each string is a vnet. _dcDao.addVnet(network.getDataCenterId(), network.getId(), addVnetsFinal); } if (removeVnetsFinal != null) { <LOG> //deleteVnets takes a list of strings to be removed. each string is a vnet. _datacneterVnet.deleteVnets(TransactionLegacy.currentTxn(), network.getDataCenterId(), network.getId(), removeVnetsFinal); } _physicalNetworkDao.update(network.getId(), network); }<log_message> ""Adding vnet range "" "" for the physicalNetwork id= "" "" and zone id="" "" as a part of updatePhysicalNetwork call"" </log_message>  <similarity> 0.89 </similarity><log_message> ""removing vnet range "" "" for the physicalNetwork id= "" "" and zone id="" "" a

In [ ]:
#CodeBLEU 점수 측정
prediction_files = tf.io.gfile.glob(prediction_files_pattern)

latest_prediction_file = sorted(
    prediction_files, key=_prediction_file_to_ckpt)[-1]

print("사용할 예측 결과 파일:", latest_prediction_file)

finetuning_task = t5.data.TaskRegistry.get(TASK_NAME)
ds = finetuning_task.get_dataset(
    split="test",
    sequence_length={"inputs": 1024, "targets": 1024},
    shuffle=False
)

results = []
with tf.io.gfile.GFile(latest_prediction_file) as preds:
    for ex, pred in zip(tfds.as_numpy(ds), preds):
        input_text = tf.compat.as_text(ex["inputs_pretokenized"])
        target_text = tf.compat.as_text(ex["targets_pretokenized"])
        prediction_text = pred.strip()
        results.append((input_text, target_text, prediction_text))

inputs, references, predictions = zip(*results)

inputs = list(inputs)
references = list(references)
predictions = list(predictions)

#def tokenize_code(code):
#    return nltk.word_tokenize(code)

def tokenize_code(code, lang='java'):
    lexer = get_lexer_by_name(lang)
    tokens = lex(code, lexer)
    token_list = [t[1] for t in tokens if t[0] in Token]
    return token_list


smooth_fn = SmoothingFunction().method1
bleu_scores = {'BLEU-1': [], 'BLEU-2': [], 'BLEU-3': [], 'BLEU-4': []}

for ref, pred in zip(references, predictions):
    ref_tokens = [tokenize_code(ref)]
    pred_tokens = tokenize_code(pred)

    bleu1 = sentence_bleu(ref_tokens, pred_tokens, weights=(1, 0, 0, 0), smoothing_function=smooth_fn)
    bleu2 = sentence_bleu(ref_tokens, pred_tokens, weights=(0.5, 0.5, 0, 0), smoothing_function=smooth_fn)
    bleu3 = sentence_bleu(ref_tokens, pred_tokens, weights=(0.33, 0.33, 0.33, 0), smoothing_function=smooth_fn)
    bleu4 = sentence_bleu(ref_tokens, pred_tokens, weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=smooth_fn)

    bleu_scores['BLEU-1'].append(bleu1)
    bleu_scores['BLEU-2'].append(bleu2)
    bleu_scores['BLEU-3'].append(bleu3)
    bleu_scores['BLEU-4'].append(bleu4)

print("==== BLEU Scores ====")
for key in bleu_scores:
    average_score = sum(bleu_scores[key]) / len(bleu_scores[key])
    print(f'{key}: {average_score:.4f}')




사용할 예측 결과 파일: gs://log_gen/finetuned-model/multi-log-injection/one-to-n/custom/word_fixed/test_eval/log_injection_600000_predictions
==== BLEU Scores ====
BLEU-1: 0.8288
BLEU-2: 0.7673
BLEU-3: 0.6907
BLEU-4: 0.6491


In [ ]:
#METEOR 점수 정
def tokenize_code(code):
    return nltk.word_tokenize(' '.join(code.rstrip('\n')[0:-1].split()).strip('"'))

meteor_scores = []
meteor_s=0.0
for ref, pred in zip(references, predictions):
    ref_tokens = tokenize_code(ref)
    pred_tokens = tokenize_code(pred)
    #score = nltk.translate.meteor_score.meteor_score([ref], pred )
    score = nltk.translate.meteor_score.meteor_score([ref_tokens], pred_tokens )
    #meteor_scores.append(score)
    meteor_s += score

average_meteor = meteor_s / len(predictions)
print(f'\nMETEOR: {average_meteor:.4f}')


METEOR: 0.9103


In [ ]:
#ROUGE-LCS Score 측정
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=False)

rouge_precision = []
rouge_recall = []
rouge_fmeasure = []

for ref, pred in zip(references, predictions):
    scores = scorer.score(pred, ref)
    rouge_l = scores['rougeL']
    rouge_precision.append(rouge_l.precision)
    rouge_recall.append(rouge_l.recall)
    rouge_fmeasure.append(rouge_l.fmeasure)

average_precision = sum(rouge_precision) / len(rouge_precision)
average_recall = sum(rouge_recall) / len(rouge_recall)
average_fmeasure = sum(rouge_fmeasure) / len(rouge_fmeasure)

print("\n==== ROUGE-LCS Scores ====")
print(f'Precision: {average_precision:.4f}')
print(f'Recall: {average_recall:.4f}')
print(f'F-measure: {average_fmeasure:.4f}')

INFO:absl:Using default tokenizer.



==== ROUGE-LCS Scores ====
Precision: 0.9140
Recall: 0.9207
F-measure: 0.9142


In [ ]:
lang = 'java'
import CodeBLEU_2

print("\n==== Summary ====")
for key in bleu_scores:
    average_score = sum(bleu_scores[key]) / len(bleu_scores[key])
    print(f'{key}: {average_score:.4f}')
print(f'METEOR: {average_meteor:.4f}')
print(f'ROUGE-L Precision: {average_precision:.4f}')
print(f'ROUGE-L Recall: {average_recall:.4f}')
print(f'ROUGE-L F-measure: {average_fmeasure:.4f}')
#print(f'CodeBLEU: {codebleu_score:.4f}')


identical_count = sum([1 for ref, pred in zip(references, predictions) if ref == pred])
total_count = len(references)

print(f"예측값과 레퍼런스가 동일한 개수: {identical_count}/{total_count}")


==== Summary ====
BLEU-1: 0.9123
BLEU-2: 0.8997
BLEU-3: 0.8893
BLEU-4: 0.8783
METEOR: 0.9123
ROUGE-L Precision: 0.9207
ROUGE-L Recall: 0.9140
ROUGE-L F-measure: 0.9142
예측값과 레퍼런스가 동일한 개수: 17174/61258


In [ ]:
!pip install javalang

import csv
import pandas as pd
import os
import logging
import subprocess
import re
import javalang
from typing import List, Dict
import xml.etree.ElementTree as ET

In [ ]:
# try-catch, system call, api 추출 방법
logging.basicConfig(level=logging.INFO)

system_call_variables=set()
API_PACKAGES = {'org', 'com', 'net'}
EXCLUDED_LIBRARIES = {'junit','slf4j', 'log4j', 'logback','mavenlogger','log','logoutputstream','logger'} #,'org.apache.maven'


def extract_variables(line, id):
    lines = line.strip()
    if not lines:
        return None, None

    if lines.startswith('//') or lines.startswith('/*') or lines.startswith('*') or lines.endswith('*/'):
        return None,None

    try:
        tokens = list(javalang.tokenizer.tokenize(lines))
        if not tokens:
            return None, None

        parser = javalang.parser.Parser(tokens)
        node = None
        try:
            node = parser.parse_statement()
            #node = parser.parse_local_variable_declaration_statement()
        except:
            parser = javalang.parser.Parser(tokens)
            node = parser.parse_member_declaration()

        if node is None:
            return None, None

        if isinstance(node, (javalang.tree.VariableDeclaration, javalang.tree.FieldDeclaration)):
            type_node = node.type
            #if isinstance(type_node, javalang.tree.ReferenceType):
            type_name = type_node.name
            print(type_name, id_list)
            if type_name in id_list:
                for declarator in node.declarators:
                    variable_name = declarator.name
                    print(f"Extracted variable: {variable_name} of type {type_name}")
                    return type_name, variable_name


        return None, None

    except (javalang.parser.JavaSyntaxError, javalang.tokenizer.LexerError, IndexError, StopIteration) and Exception as e:
        return None, None

def is_variable_used(line, variable_name):
    usage_pattern = r'\b{}\b'.format(re.escape(variable_name))
    return re.search(usage_pattern, line) is not None

def is_real_system_call(line):
    lines = line.strip()
    if not lines:
        return False

    if lines.startswith('//') or lines.startswith('/*') or lines.startswith('*') or lines.endswith('*/'):
        return False

    try:
        tokens = list(javalang.tokenizer.tokenize(lines))
        if not tokens:
            return False

        parser = javalang.parser.Parser(tokens)
        try:
            node = parser.parse_statement()
        except:
            parser = javalang.parser.Parser(tokens)
            #node = parser.parse_expression()
            node = parser.parse_member_declaration()

        system_call_found = False

        def get_type_name(type_node):
            if isinstance(type_node, javalang.tree.ReferenceType):
                type_name = type_node.name
                if type_node.arguments:
                    for arg in type_node.arguments:
                        if isinstance(arg, javalang.tree.TypeArgument):
                            arg_type_name = get_type_name(arg.type)
                            if arg_type_name:
                                return arg_type_name
                return type_name
            elif isinstance(type_node, javalang.tree.BasicType):
                return type_node.name.lower()
            return None

        def traverse(node):
            nonlocal system_call_found

            ignored_node_types = (
                javalang.tree.MethodDeclaration,
                javalang.tree.ClassDeclaration,
                javalang.tree.Literal
            )
            if isinstance(node, ignored_node_types):
                return


            if isinstance(node, javalang.tree.FormalParameter):
                param_type = get_type_name(node.type)
                if param_type and param_type in system_call_patterns_lower:
                    variable_name = node.name
                    system_call_variables.add(variable_name)
                    #system_call_found = True
                    return

            if isinstance(node, (javalang.tree.VariableDeclaration, javalang.tree.FieldDeclaration)):
                type_node = node.type #if hasattr(node, 'type') else None
                type_name = type_node.name
                if type_name in system_call_patterns:
                    for decl in node.declarators:
                        variable_name = decl.name
                        system_call_variables.add(variable_name)
                    system_call_found = True
                    return

            # if isinstance(node, javalang.tree.MemberReference):
            #     variable_name = node.member
            #     if variable_name in system_call_variables:
            #         system_call_found = True
            #         return

                    #if type_name in system_call_patterns_lower:
                    #    system_call_found = True
                    #    return

            if isinstance(node, javalang.tree.MethodInvocation):

                method_name = node.member.lower()
                if method_name in system_call_patterns_lower:
                    system_call_found = True
                    return

            # if isinstance(node, javalang.tree.ClassCreator):
            #     if hasattr(node, 'type') and node.type:
            #         type_name = node.type.name.lower()
            #         if type_name in system_call_patterns_lower:
            #             system_call_found = True
            #             return

            for child in node.children:
                if isinstance(child, (list, tuple)):
                    for item in child:
                        if isinstance(item, javalang.ast.Node):
                            traverse(item)
                elif isinstance(child, javalang.ast.Node):
                    traverse(child)

        traverse(node)

        return system_call_found

    except (javalang.parser.JavaSyntaxError, IndexError, StopIteration, TypeError) and Exception as e:
        return False

def is_method_declaration(line):
    method_decl_patterns = ["void", "public", "protected", "private", "static", "final", "abstract", "synchronized"]
    return any(line.strip().lower().startswith(keyword) for keyword in method_decl_patterns)

def is_variable_declaration_or_assignment(line):
    var_decl_patterns = [r"\bint\b", r"\bfloat\b", r"\bdouble\b", r"\bchar\b", r"\bstring\b", r"\bboolean\b",
                         r"\blong\b", r"\bbyte\b", r"\bshort\b"]
    return any(re.search(pattern, line, re.IGNORECASE) for pattern in var_decl_patterns)


def extract_and_insert_token( method, api_short_patterns):
    modified_methods=[]
    system_call_variables.clear()
    system_call_found = False
    try_catch_found = False
    api_found = False
    api_variable_found= False
    system_call_variable_found = False
    check = False
    api_variables = []
    #system_call_variables=[]
    code = method.strip()
    if code:
        lines = code.splitlines()
        print(len(lines))
        if len(lines) <= 1:
          return 0
        modified_lines = []
        i = 0

        in_multiline_comment = False


        for line in lines:
            if len(api_short_patterns) > 0:
                for api_short in api_short_patterns:
                    api_name, variable_name = extract_variables(line, api_short)
                    if api_name and variable_name:
                        if variable_name not in api_variables:
                            api_variables.append(variable_name)

            for system_call in system_call_patterns:
                sys_name, variable_name = extract_variables(line,system_call)
                if sys_name and variable_name:
                    system_call_variables.add(variable_name)

        while i <len(lines):
            line = lines[i]
            strip_line = line.strip()

            if strip_line.startswith("/*"):
                in_multiline_comment = True
                modified_lines.append(line)
                i += 1
                continue

            if in_multiline_comment:
                modified_lines.append(line)
                if '*/' in strip_line:
                    in_multiline_comment = False
                i += 1
                continue

            if strip_line.startswith("//"):
                #modified_lines.append(line)
                i += 1
                continue

            if "try" in strip_line.lower() and "{" in strip_line:
                if re.search(r'try\s*{', code) and re.search(r'catch\s*\(', code):
                    try_catch_found = True

            if is_real_system_call(line):
                system_call_found = True
                while i < len(lines):
                    modified_lines.append(lines[i])
                    if lines[i].strip().endswith(";"):
                        i += 1
                        modified_lines.append("<LOG>")
                        break
                    i += 1
                continue

            if any(api_short in strip_line for api_short in api_short_patterns):
                api_found = True
                while i < len(lines):
                    modified_lines.append(lines[i])
                    if lines[i].strip().endswith(";"):
                        i += 1
                        modified_lines.append("<LOG>")
                        break
                    i += 1
                continue

            api_variable_used = False
            for variable in api_variables:
                if is_variable_used(line, variable):
                    api_variable_found = True
                    modified_lines.append(line)
                    modified_lines.append("<LOG>")
                    api_variable_used = True
                    i += 1
                    break
            if api_variable_used:
                continue

            system_call_variable_used = False
            for variable in system_call_variables:
                if is_variable_used(line, variable):
                    system_call_variable_found = True
                    modified_lines.append(line)
                    modified_lines.append("<LOG>")
                    system_call_variable_used = True
                    i += 1
                    break
            if system_call_variable_used:
                continue

            if "catch" in strip_line.lower() and "{" in strip_line:
                if re.search(r'catch\s*\(', strip_line):
                    try_catch_found = True
                    modified_lines.append(line)
                    modified_lines.append("<LOG>")
                    i += 1
                    continue

            modified_lines.append(line)
            i+=1

        modified_lines = '\n'.join(modified_lines)
        if system_call_found == True or api_found == True or try_catch_found==True:
            check = True
        method_info = {
            "code": modified_lines,
            "check": check,
        }
        modified_methods.append(method_info)
    return modified_methods

def format_java_code(code):
    formatted_code = ""
    indent_level = 0

    for line in code.replace("{", "{\n").replace("}", "\n}").replace(";", ";\n").splitlines():
        line = line.strip()

        if line.startswith("}"):
            indent_level -= 1

        formatted_code += "\t" * indent_level + line

        if line.endswith("{"):
            indent_level += 1
    return formatted_code

def flatten_code(code):
    flattened_code = ' '.join(line.strip() for line in code.splitlines())
    return flattened_code

In [ ]:
#from transformers import T5Tokenizer, T5ForConditionalGeneration
import tempfile
import os
import gc
output_file1= "/content/ex1.txt"
output_file2= "/content/ex2.txt"
output_file3= "/content/ex3.txt"

def get_prediction_token(input_text, model_dir): #method에 system call, api, try catch가 있는 경우
    model = t5.models.MtfModel(FINETUNE_MODEL_DIR, tpu='local',sequence_length={"inputs": 1024, "targets": 2048}, learning_rate_schedule = learning_rate_scheduler)

    with tempfile.NamedTemporaryFile(delete=False, mode='w', encoding='utf-8') as tmp_input_file:
        tmp_input_file.write(input_text)
        input_file_path = tmp_input_file.name

    tmp_output_file = tempfile.NamedTemporaryFile(delete=False)
    output_file_path = tmp_output_file.name
    tmp_output_file.close()
    vocabulary = t5.data.get_default_vocabulary()
    model.predict(input_file=input_file_path, output_file=output_file1, vocabulary= load_vocabulary(),)

    with open('ex1.txt-600000', 'r', encoding='utf-8') as f:
        prediction = f.read().strip()
        print(f.read())
    os.remove(input_file_path)
    #os.remove('/content/ex1.txt-600000')
    return prediction

def get_prediction_location(input_text, model_dir): #method에 system call, api, try catch가 없는 경우 로그 삽입 여부 결정
    model = t5.models.MtfModel(model_dir, tpu='local',sequence_length={"inputs": 1024, "targets": 2048}, learning_rate_schedule = learning_rate_scheduler)

    with tempfile.NamedTemporaryFile(delete=False, mode='w', encoding='utf-8') as tmp_input_file:
        tmp_input_file.write(input_text)
        input_file_path = tmp_input_file.name

    # Generate the prediction
    vocabulary = t5.data.get_default_vocabulary()
    model.predict(input_file=input_file_path, output_file=output_file2, vocabulary= load_vocabulary())
    #output_file= "/content/ex1.txt-600000"
    with open('/content/ex2.txt-330000', 'r') as f:#, encoding='utf-8') as f:
        prediction = f.read().strip()
        print(f.read())

    os.remove(input_file_path)
    #os.remove('/content/ex2.txt-330000')

    return prediction

def get_prediction_no_token(input_text, model_dir):#method에 system call, api, try catch가 없는 경우 로그 메세지 생성
    model = t5.models.MtfModel(model_dir, tpu='local',sequence_length={"inputs": 1024, "targets": 2048}, learning_rate_schedule = learning_rate_scheduler)

    with tempfile.NamedTemporaryFile(delete=False, mode='w', encoding='utf-8') as tmp_input_file:
        tmp_input_file.write(input_text)
        input_file_path = tmp_input_file.name

    # Generate the prediction
    vocabulary = t5.data.get_default_vocabulary()
    model.predict(input_file=input_file_path, output_file=output_file3, vocabulary= load_vocabulary())
    #output_file= "/content/ex1.txt-600000"
    with open('/content/ex3.txt-600000', 'r') as f:#, encoding='utf-8') as f:
        prediction = f.read().strip()
        print(f.read())

    os.remove(input_file_path)
    #os.remove('/content/ex3.txt-600000')

    return prediction

if __name__ == "__main__":
    modified_methods = extract_and_insert_token(method, api_short_patterns)
    if modified_methods[0]['check'] == True:
      print("System Call, Try Catch, API Yes")
      input_method = flatten_code(modified_methods[0]['code'])
      prediction = get_prediction_token(input_method, FINETUNE_MODEL_DIR)
      print("Original Input: ", format_java_code(method))
      print("Preprocess Input:", format_java_code(input_method))
      print("Prediction:", format_java_code(prediction))
    else:
      print("System Call, Try Catch, API No")
      input_method = method
      model_directory = f'gs://log_gen/finetuned-model/classifier/polynomial2'
      prediction_whether_log_need = get_prediction_location(input_method, model_directory)
      print(prediction_whether_log_need)
      if prediction_whether_log_need == "Need":
        print("Log Need")
        model_directory2 = f'gs://log_gen/finetuned-model/multi-log-injection/one-to-n/custom/word'
        prediction = get_prediction_no_token(input_method, model_directory2)
        print("Input:", format_java_code(input_method))
        print("Prediction:", format_java_code(prediction))
      else:
        print("Log Not Need")
        print(format_java_code(input_method))


    del model
    gc.collect()

INFO:root:system_path_file_exists:gs://log_gen/finetuned-model/multi-log-injection/one-to-n/custom/word_fixed/operative_config.gin
ERROR:root:Path not found: gs://log_gen/finetuned-model/multi-log-injection/one-to-n/custom/word_fixed/operative_config.gin


8
System Call, Try Catch, API Yes
